In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
import datetime

#### please install the pygeocoder package before running this code.
<b>Follow these steps the install package:</b><br>
1) unzip pygeocoder-1.2.5.zip.<br>
2) in CMD, cd to: ../pygeocoder-1.2.5/dist/pygeocoder-1.2.5<br>
3) run: python setup.py install<br>

In [12]:
from pygeocoder import Geocoder

Put your <b>NAME</b> and the <b>PATCH</b> number below:

In [13]:
# names: Adil , Mohammad , Asim
# Pick a patch number from Patches.xlsx file in shared folder, and write your name:
# You can run the code as many times as needed, it will always pick up from when it left per patch.

name = 'Adil'
patch = 10

# Performane parameters (no need to change unless the code is exiting fast)
sleep_time = 0.15
trials = 30

# RUN ALL CELLS BELOW

In [14]:
p1 = patch * 1000
p2 = (patch+1)*1000

In [15]:
floc = name+'_coordinates_'+repr(p1)+'_'+repr(p2-1)+'.csv'
data_file = Path(floc)

if not data_file.is_file():
    calls = pd.read_csv('911_calls_for_service.csv')
    calls = calls.drop('Unnamed: 0',axis=1)
    data = calls.loc[(calls.location =='(,)') | (calls.location =='(')].copy()
    data = data.iloc[p1:p2]
    data.location = data.location.replace(to_replace='(',value='(,)')
    data.incidentLocation = data.incidentLocation.apply(lambda x: str(x).replace('/',' '))
    data.to_csv(floc)

data = pd.read_csv(floc)
data = data.drop('Unnamed: 0',axis=1)
data.incidentLocation = data.incidentLocation.apply(lambda x: str(x).replace('/',' '))
    
ogn_before = data.loc[data.location =='(,)','location'].count()

In [16]:
def get_coordinates(addr):
    try:
        time.sleep(sleep_time)
        result = Geocoder.geocode(addr + ' Baltimore')
        if (len(result.coordinates) > 0) & (result.city == 'Baltimore'):
            return result.coordinates
        else:
            return np.nan
    except Exception as e:
        if str(e).splitlines()[0] == 'Error ZERO_RESULTS':
            return np.nan
        else:
            #print(str(e))
            return 'ERROR'

In [17]:
%%time

print('Start time:', str(datetime.datetime.now()))
print("Patch: "+repr(p1)+'-'+repr(p2-1))
print("Starting missing: ",ogn_before,'\n')

before=ogn_before
trial = 0

while trial < trials:
    #time.sleep(1)
    after = 0
    print("--- Trial", trial+1,"---")
    while before-after > 0:
        before = data.loc[data.location =='(,)','location'].count()
        for x in range(len(data)):
            if (data.iloc[x,6] == '(,)'):
                c = get_coordinates(data.iloc[x,5].strip())
                if c == 'ERROR':
                    break
                else:
                    data.iloc[x,6] = str(c)
        after = data.loc[data.location =='(,)','location'].count()
        data.to_csv(floc)
        print('T['+repr(trial+1)+']:','Resolved: ',before-after, 'Total: ',ogn_before-after)
    trial+=1

Start time: 2017-12-04 21:18:19.076274
Patch: 10000-10999
Starting missing:  0 

--- Trial 1 ---
--- Trial 2 ---
--- Trial 3 ---
--- Trial 4 ---
--- Trial 5 ---
--- Trial 6 ---
--- Trial 7 ---
--- Trial 8 ---
--- Trial 9 ---
--- Trial 10 ---
--- Trial 11 ---
--- Trial 12 ---
--- Trial 13 ---
--- Trial 14 ---
--- Trial 15 ---
--- Trial 16 ---
--- Trial 17 ---
--- Trial 18 ---
--- Trial 19 ---
--- Trial 20 ---
--- Trial 21 ---
--- Trial 22 ---
--- Trial 23 ---
--- Trial 24 ---
--- Trial 25 ---
--- Trial 26 ---
--- Trial 27 ---
--- Trial 28 ---
--- Trial 29 ---
--- Trial 30 ---
Wall time: 2 ms


In [18]:
print("\nPatch: "+repr(p1)+'-'+repr(p2-1))
print("Starting missing: ",ogn_before)
print("Ending missing: ", after)
print("Resolved coordinates: ",ogn_before-after)
print('Success rate: %', (1-data.location[data.location.isnull()==True].size/len(data))*100)


Patch: 10000-10999
Starting missing:  0
Ending missing:  0
Resolved coordinates:  0
Success rate: % 92.10000000000001


In [19]:
data.loc[data.location =='(,)','location'].count()

0

In [20]:
data.location.isnull().value_counts()

False    921
True      79
Name: location, dtype: int64

In [21]:
#data.loc[data.location !='(,)'].tail()